In [ ]:
import aurigaia as ag  # Local package, manually pip install this!
import aurigaia.kapteyn as ak

import numpy as np
import matplotlib.pyplot as plt

## Functions

In [ ]:
from math import radians, pi
from aurigaia.read_aurigaia import BaseFilter


class PositionFilter(BaseFilter):
    """
    Filters the stars based on given intervals for the Ra and Dec:
    ra_range:  [ra_min,  ra_max]
    dec_range: [dec_min, dec_max]
    
    The range is inclusive.
    
    `deg` specifies the unit of the input ranges.
    Assumed intervals are:
        ra:  [-pi,   pi]
        dec: [-pi/2, pi/2]
    
    WARNING: providing an interval like (for ra): (170, 190) will be invalid,
                due to the interval of the database. It will wrap to (170, -170)
                and thus give something invalid. The function should be adapted
                to properly handle these intervals as well.
    """
    def __init__(self, ra_range=None, dec_range=None, obs=False, deg=True):
        self.quantity = "HCoordinates"
        if obs:
            self.quantity += "Obs"

        self.ra_min, self.ra_max = ra_range
        self.dec_min, self.dec_max = dec_range

        # Convert degrees to radians if needed
        if deg:
            self.ra_min = radians(self.ra_min)
            self.ra_max = radians(self.ra_max)
            self.dec_min = radians(self.dec_min)
            self.dec_max = radians(self.dec_max)

        # Change interval to be in [-pi , pi] and [-pi/2, pi/2] for ra and dec
        # respectively
        if self.ra_min > pi:
            self.ra_min -= 2 * pi
        if self.ra_max > pi:
            self.ra_max -= 2 * pi
        if self.dec_min > pi / 2:
            self.dec_min -= pi
        if self.dec_max > pi / 2:
            self.dec_max -= pi

    def stars_to_keep(self, nstars, data):
        ra = data[self.quantity][:, 0]
        dec = data[self.quantity][:, 1]
        return (self.ra_min <= ra) & (ra <= self.ra_max) & (self.dec_min <= dec) & (dec <= self.dec_max)

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

def calculate_galactic_coords(hcoordinates, hvelocities):
   # Extract coordinate components from the HCoordinates dataset
    ra = u.Quantity(hcoordinates[:, 0], unit=u.radian)
    dec = u.Quantity(hcoordinates[:, 1], unit=u.radian)
    parallax = u.Quantity(hcoordinates[:, 2], unit=u.arcsec)

    # Extract velocities from HVelocities dataset
    pm_ra_cosdec = u.Quantity(hvelocities[:, 0], unit=u.arcsec / u.year)
    pm_dec = u.Quantity(hvelocities[:, 1], unit=u.arcsec / u.year)
    rv = u.Quantity(hvelocities[:, 2], unit=u.km / u.second)

    # Calculate distance to each star:
    # Distance in parsecs is just 1.0/(parallax in arcsecs),
    # but here we let astropy deal with the units.
    dist = parallax.to(u.kpc, equivalencies=u.parallax())

    # Translate to galactic coordinates using astropy
    coords = SkyCoord(ra=ra, dec=dec, distance=dist,
                  pm_ra_cosdec=pm_ra_cosdec, pm_dec=pm_dec,
                  radial_velocity=rv, frame='icrs')
    gal = coords.galactic
    
    # New shape is (#stars, 2)
    shape = (hcoordinates.shape[0], 2)
    
    # Put the new coordinates into fresh arrays
    pos = np.empty(shape)
    pos[:, 0] = gal.l.deg
    pos[:, 1] = gal.b.deg
    
    vel = np.empty(shape)
    vel[:, 0] = gal.pm_l_cosb.to(u.arcsec / u.yr).value
    vel[:, 1] = gal.pm_b.to(u.arcsec / u.yr).value
    
    return pos, vel

In [ ]:
import h5py
from time import process_time

def save_hdf5(data, halo_nr, ext=True, prefix=None):
    ext_str = "ext" if ext else "noext"
    
    if prefix is None:
        prefix = ""
    
    data_filename = f"{prefix}ICC_{ext_str}_Au{halo_nr:02d}.hdf5"
    print(f"Writing data to {data_filename}...")
    
    start = process_time()
    
    with h5py.File(data_filename, 'w') as f:
        # Add everything with a single column
        for k, v in data["stars"].items():
            if len(v.shape) == 1:
                f.create_dataset(k, data=v)

        # Assumes Pos, Vel, and the HCoordinate/ HVelocities are available
        for i, l in enumerate(["X", "Y", "Z"]):
            f.create_dataset(f"Pos{l}", data=data["stars"]["Pos"][:, i])
            f.create_dataset(f"Vel{l}", data=data["stars"]["Vel"][:, i])
            f.create_dataset(f"Pos{l}Obs", data=data["stars"]["PosObs"][:, i])
            f.create_dataset(f"Vel{l}Obs", data=data["stars"]["VelObs"][:, i])

        for i, l in enumerate(["RA", "Dec", "Parallax"]):
            f.create_dataset(f"{l}", data=data["stars"]["HCoordinates"][:, i])
            f.create_dataset(f"{l}Obs", data=data["stars"]["HCoordinatesObs"][:, i])
            f.create_dataset(f"{l}Error", data=data["stars"]["HCoordinateErrors"][:, i])
            
            name = f"PM{l}"
            if l == "Parallax":
                name = "RadialVelocity"
            f.create_dataset(name, data=data["stars"]["HVelocities"][:, i])
            f.create_dataset(name + "Obs", data=data["stars"]["HVelocitiesObs"][:, i])
            f.create_dataset(name + "Error", data=data["stars"]["HVelocityErrors"][:, i])
            
    end = process_time()
    print(f"Writing took {end - start}s")
    
    metadata_filename = f"{prefix}ICC_{ext_str}_Au{halo_nr:02d}-meta.hdf5"
    print(f"Writing metadata to {metadata_filename}...")
    start = process_time()
    
    # Save the metadata separately, since vaex doesn't know how to read it if
    # it is included.
    with h5py.File(metadata_filename, 'w') as f:
        # Parameters
        param = f.create_group("Parameters")
        for k, v in data["Parameters"].items():
            param[k] = v

        # Headers
        head = f.create_group("Header")
        for k, v in data["Header"].items():
            head[k] = v

    end = process_time()
    print(f"Writing took {end - start}s")

In [ ]:
from copy import deepcopy

# Filter out everything outside 24x24 box
def select_box(data, copy=True):
    newdata = data
    if copy:
        newdata = deepcopy(newdata)
    
    l = data["stars"]["l_wrap"]
    b = data["stars"]["b"]
    selection = (l > -12) & (l < 12) & (b > -12) & (b < 12)
    for k, v in data["stars"].items():
        newdata["stars"][k] = v[selection, ...]
    return newdata

In [ ]:
from aurigaia.read_aurigaia import RangeFilter, DifferenceFilter


def grab_halo(halo_nr, ext=True, location_prefix=None, verbose=True):
    # Create filters
    pos_filt = PositionFilter(ra_range=(247, 285), dec_range=(-46, -12))
    range_filt = RangeFilter('Gmagnitude', vmin=13.5, vmax=16.5)

    filters = [range_filt, pos_filt]

    # Read the data from gaia2 server
    data = ak.kapteyn_aurigaia(halo_n=halo_nr,
                                   angle=30,
                                   ext=ext,
                                   filters=filters,
                                   verbose=verbose,
                                   datasets=ak.available_props,
                                   units=False)
    
    # Add galactic coordinates as well
    pos, vel = calculate_galactic_coords(data["stars"]["HCoordinates"], data["stars"]["HVelocities"])
    data["stars"]["l"] = pos[:, 0]
    data["stars"]["l_wrap"] = np.where(pos[:, 0] > 180, pos[:, 0] - 360, pos[:, 0])
    data["stars"]["b"] = pos[:, 1]
    data["stars"]["PMl_cosb"] = vel[:, 0]
    data["stars"]["PMb"] = vel[:, 1]
    
    pos, vel = calculate_galactic_coords(data["stars"]["HCoordinatesObs"], data["stars"]["HVelocitiesObs"])
    data["stars"]["lObs"] = pos[:, 0]
    data["stars"]["l_wrapObs"] = np.where(pos[:, 0] > 180, pos[:, 0] - 360, pos[:, 0])
    data["stars"]["bObs"] = pos[:, 1]
    data["stars"]["PMl_cosbObs"] = vel[:, 0]
    data["stars"]["PMbObs"] = vel[:, 1]
    
    # Remove everything out 24x24 box in place
    select_box(data, copy=False)
    
    # And save the data
    save_hdf5(data, halo_nr=halo_nr, ext=ext, prefix="./data/mocks/")

## Actually Running

In [ ]:
AVAIL_HALO = [6, 16, 21, 23, 24]
# TODO: halo 27 only noext available
grab_halo(6, ext=True, location_prefix="./data/mocks/")

## Old Manual Running

In [ ]:
ak.available_props

In [ ]:
from aurigaia.read_aurigaia import RangeFilter, DifferenceFilter
pos_filt = PositionFilter(ra_range=(247, 285), dec_range=(-46, -12))
range_filt = RangeFilter('Gmagnitude', vmin=13.5, vmax=16.5)

filters = [range_filt, pos_filt]

In [ ]:
data_ext = ak.kapteyn_aurigaia(halo_n=6,
                           angle=30,
                           ext=True,
                           filters=filters,
                           verbose=True,
                           datasets=ak.available_props,
                           units=False)

In [ ]:
data_ext["stars"]["HVelocities"].shape

In [ ]:
pos, vel = calculate_galactic_coords(data_ext["stars"]["HCoordinates"], data_ext["stars"]["HVelocities"])
data["stars"]["l"] = pos[:, 0]
data["stars"]["l_wrap"] = np.where(pos[:, 0] > 180, pos[:, 0] - 360, pos[:, 0])
data["stars"]["b"] = pos[:, 1]
data["stars"]["PMl_cosb"] = vel[:, 0]
data["stars"]["PMb"] = vel[:, 1]

In [ ]:
pos, vel = calculate_galactic_coords(data_ext["stars"]["HCoordinatesObs"], data_ext["stars"]["HVelocitiesObs"])
data["stars"]["lObs"] = pos[:, 0]
data["stars"]["l_wrapObs"] = np.where(pos[:, 0] > 180, pos[:, 0] - 360, pos[:, 0])
data["stars"]["bObs"] = pos[:, 1]
data["stars"]["PMl_cosbObs"] = vel[:, 0]
data["stars"]["PMbObs"] = vel[:, 1]

In [ ]:
x = np.arange(100)
x.shape = (10, 10)
x[0:3, :]

In [ ]:
save_hdf5(data_ext, halo_nr=6, ext=True, prefix="./data/mocks/")

In [ ]:
# tryout filter l
dat = select_box(data_ext, copy=True)
save_hdf5(dat, halo_nr=6, ext=True, prefix="./data/mocks/")

## Verifying data correctness

In [ ]:
import vaex

In [ ]:
f = vaex.open("./data/mocks/ICC_ext_Au06.hdf5")
f.info()

In [ ]:
f.viz.heatmap('PosX', 'PosY', what='count(*)', f='log10', selection='Parallax < 0.00025')

In [ ]:
selection = "Parallax < 0.00025"
fig, axs = plt.subplots(1, 2, figsize=(11, 5))

ax = axs[0]
plt.sca(ax)
f.viz.heatmap('l_wrap', 'b', what='count(*)', f='log1p', selection=selection)
ax.vlines(-12, -12, 12, ls='--', color='k')
ax.vlines(12, -12, 12, ls='--', color='k')
ax.hlines(-12, -12, 12, ls='--', color='k')
ax.hlines(12, -12, 12, ls='--', color='k')
ax.set_xlabel('l (deg)')
ax.set_ylabel('b (deg)')
ax.invert_xaxis()

ax = axs[1]
plt.sca(ax)
f.viz.heatmap('l_wrap', 'b', what='count(*)', f='log1p', limits=[-12, 12], selection=selection)
ax.set_xlabel('l (deg)')
ax.set_ylabel('b (deg)')
ax.invert_xaxis()

plt.show()

In [ ]:
f['Distance'] = 1 / f['Parallax']
f['GabsMagnitude'] = f['Gmagnitude'] - 5*np.log10(f['Distance'] / 10)

# Deredden
VtoG = 0.718
VtoBP = 0.9875
VtoRP = 0.576

f["GabsMagnitudeDR"] = f["GabsMagnitude"] - f["Extinction31"]*VtoG
f["GBmagnitudeDR"] = f["GBmagnitude"] - f["Extinction31"]*VtoBP
f["GRmagnitudeDR"] = f["GRmagnitude"] - f["Extinction31"]*VtoRP

In [ ]:
box_selection = "(l_wrap > -12) & (l_wrap < 12) & (b > -12) & (b < 12) & (Parallax < 0.00025)"

fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs = axs.flatten()

ax = axs[0]
plt.sca(ax)
f.viz.heatmap('GBmagnitude - GRmagnitude', '-GabsMagnitude', f='log10', colormap='YlOrBr', selection=box_selection)
ax.set_title('Actual')

ax = axs[1]
plt.sca(ax)
f.viz.heatmap('GBmagnitudeDR - GRmagnitudeDR', '-GabsMagnitudeDR', f='log10', colormap='YlOrBr', selection=box_selection)
ax.set_title('Dereddened')

plt.show()